In [ ]:
import socket ,cv2, pickle ,struct

server_socket = socket.socket (socket.AF_INET ,socket.SOCK_STREAM)#server socket created to send data

c_socket = socket.socket (socket.AF_INET ,socket.SOCK_STREAM)#client socket created to accept data

server_socket.bind(('192.168.3.100',8888))#socket become server to send video data with server ip and port
server_socket.listen(5)
c_socket.connect(('192.168.3.103',8888))#socket become client to receive video data with client ip and port
data = b"" # initialize data as bytes
payload_size =struct.calcsize("Q")# unsigned Q take long integer of 8 bytes
while True:
    client_socket ,addr = server_socket.accept()#accepting client data
    print('GOT CONNECTION FROM : ',addr)
    
    vid = cv2.VideoCapture(0)#taking picture from webcam
    while (vid.isOpened()):
        img , frame = vid.read()#storing picture from webcam 
        a = pickle.dumps(frame)#converting object into bytes 
        message = struct .pack("Q",len(a))+a#interpret bytes as packed binary data
        client_socket.sendall(message)#sending packed binary data to client
    
            
        while len(data) < payload_size:
            packet = c_socket.recv(4*1024)#receiving packed binary data
            if not packet:
                break
            data += packet#storing packed binary data in data
           
        packed_msg_size = data[:payload_size]#first eight byte contain packed message
        data = data [payload_size :]#rest of byte is our data
        msg_size=struct.unpack("Q" , packed_msg_size)[0]#extracting message size
    
        while len (data) < msg_size:
            data += c_socket.recv(4*1024)
        frame_data = data[:msg_size]#extracting first frame data
        data = data [msg_size:]
        frame = pickle.loads(frame_data)#converting bytes into object
        cv2.imshow("RECEived" , frame)#displaying received data
        if cv2.waitKey(1) == 13:
            break
        
c_socket.close()

GOT CONNECTION FROM :  ('192.168.3.103', 57655)
